In [39]:
import cv2 as cv
import pandas as pd
import numpy as np
from scipy import ndimage


In [1]:
path = 'J:/Disco/Camera/192.168.0.211_80/2023/06/25/'
files = ['rec_2023_06_25_12_14_29.mp4', 'rec_2023_06_25_12_14_48.mp4', 'rec_2023_06_25_12_15_20.mp4']

In [3]:
def get_frame(video_path, frame_number):
    capture = cv.VideoCapture(video_path)
    capture.set(1, frame_number)
    ret, frame = capture.read()
    capture.release()
    return frame

def get_frames(video_path, frame_numbers):
    frames = []
    for frame_number in frame_numbers:
        frames.append(get_frame(video_path, frame_number))
    return frames

def get_number_of_frames(video_path):
    capture = cv.VideoCapture(video_path)
    num_frames = int(capture.get(cv.CAP_PROP_FRAME_COUNT))
    capture.release()
    return num_frames

def get_all_frames(video_path):
    frames = []
    frame_count = get_number_of_frames(video_path)
    for frame_number in range(frame_count):
        frames.append(get_frame(video_path, frame_number))
    return frames

In [25]:
def frequency_filter(img):
    # Perform the 2-D fast Fourier Transform on the image
    f = np.fft.fft2(img)

    # Shift the zero-frequency component to the center of the spectrum
    fshift = np.fft.fftshift(f)

    # Create a mask with the same size as the image that is True for the frequencies you want to keep
    mask = np.zeros_like(img, dtype=bool)
    mask[img.shape[0]//2-150:img.shape[0]//2+150, img.shape[1]//2-150:img.shape[1]//2+150] = True

    # Apply the mask to the frequency-domain image
    fshift_masked = fshift * mask

    # Shift the zero-frequency component back to the top left corner
    f_ishift = np.fft.ifftshift(fshift_masked)

    # Perform the inverse 2-D fast Fourier Transform
    img_back = np.fft.ifft2(f_ishift)

    # Take only the real part of the result
    img_back = np.abs(img_back)

    return img_back

In [23]:
def play_video(video_path):
    capture = cv.VideoCapture(video_path)

    # Get the framerate
    fps = capture.get(cv.CAP_PROP_FPS)

    # Calculate the frame delay (in milliseconds)
    frame_delay = int(1000 / fps)
    area_threshold = 50
    object_detector = cv.createBackgroundSubtractorMOG2(history=150, varThreshold=20)

    # Show video
    while True:
        ret, frame = capture.read()

        # If frame is read correctly, ret is True
        if not ret:
            # If we've reached the end of the video (or there's an error), start it over
            capture.set(cv.CAP_PROP_POS_FRAMES, 0)
            continue
        
        # Apply filtering
        frame_f = frequency_filter(frame)
        #print(frame_f.shape)
        #print(frame.shape)

        # Detect objects
        mask = object_detector.apply(frame_f)
        _, mask = cv.threshold(mask, 50, 255, cv.THRESH_BINARY)
        contours, _ = cv.findContours(mask, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
        for cnt in contours:
            area = cv.contourArea(cnt)
            if area > area_threshold:
                #cv.drawContours(frame, [cnt], -1, (0, 255, 0), 2)
                x,y,w,h = cv.boundingRect(cnt)
                cv.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 1)

        cv.imshow('frame', frame)
        cv.imshow('mask', mask)
        cv.imshow('filtered', frame_f)
        if cv.waitKey(frame_delay) & 0xFF == ord('q'):  # wait for frame_delay ms; break if 'q' is pressed
            break

    capture.release()
    cv.destroyAllWindows()

In [27]:
def apply_threshold(frame, threshold_value=127, max_value=255):
    # Convert the frame to grayscale
    gray_frame = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Apply thresholding
    _, thresholded_frame = cv.threshold(gray_frame, threshold_value, max_value, cv.THRESH_BINARY)

    return thresholded_frame

def play_video_with_thresholding(video_path, threshold_value=127, max_value=255):
    # Open the video file
    cap = cv.VideoCapture(video_path)

    while(cap.isOpened()):
        # Read a frame from the video
        ret, frame = cap.read()

        # If the frame was read correctly ret is True
        if not ret:
            break

        # Apply thresholding to the frame
        thresholded_frame = apply_threshold(frame, threshold_value, max_value)

        # Display the thresholded frame
        cv.imshow('frame', thresholded_frame)

        # Wait for 25 ms before moving on to the next frame
        # This will slow down the video
        cv.waitKey(25)

    # Release the video file
    cap.release()

    # Close all OpenCV windows
    cv.destroyAllWindows()

In [41]:
def directional_filter(image, angle):
    # Create a directional filter mask
    mask = np.ones(image.shape, dtype=bool)
    mask = ndimage.rotate(mask, angle, reshape=False)

    # Apply the mask to the image
    filtered_image = image * mask

    return filtered_image

def play_video_with_directional_filtering(video_path, angle):
    # Open the video file
    cap = cv.VideoCapture(video_path)

    while(cap.isOpened()):
        # Read a frame from the video
        ret, frame = cap.read()

        # If the frame was read correctly ret is True
        if not ret:
            break

        # Apply directional filtering to the frame
        filtered_frame = directional_filter(frame, angle)

        # Display the filtered frame
        cv.imshow('frame', filtered_frame)

        # Wait for 25 ms before moving on to the next frame
        # This will slow down the video
        cv.waitKey(25)

    # Release the video file
    cap.release()

    # Close all OpenCV windows
    cv.destroyAllWindows()

In [59]:
'''def remove_lines(frame, threshold=200, min_line_length=100, max_line_gap=10):
    # Convert the frame to grayscale
    #gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Apply edge detection
    edges = cv.Canny(frame, 50, 150, apertureSize=3)

    # Detect lines using the Hough transform
    lines = cv.HoughLinesP(edges, 1, np.pi/180, threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    # If lines were found
    if lines is not None:
        # Draw black lines on the original frame
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv.line(frame, (x1, y1), (x2, y2), (0, 0, 0), 2)

    return frame
'''
def remove_lines(frame, threshold=200, min_line_length=100, max_line_gap=10):
    # Convert the frame to grayscale
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Apply edge detection
    edges = cv.Canny(gray, 50, 150, apertureSize=3)

    # Detect lines using the Hough transform
    lines = cv.HoughLinesP(edges, 1, np.pi/180, threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    # Create an empty mask to draw the lines on
    mask = np.zeros_like(frame)

    # If lines were found
    if lines is not None:
        # Draw white lines on the mask
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv.line(mask, (x1, y1), (x2, y2), (255, 255, 255), 2)

    # Inpaint the original frame using the mask
    frame_inpaint = cv.inpaint(frame, cv.cvtColor(mask, cv.COLOR_BGR2GRAY), 3, cv.INPAINT_TELEA)

    return frame_inpaint


def play_video_with_line_removal(video_path, threshold_lines=200, min_line_length=100, max_line_gap=10, threshold_value=127):
    # Open the video file
    cap = cv.VideoCapture(video_path)

    while(cap.isOpened()):
        # Read a frame from the video
        ret, frame = cap.read()

        # If the frame was read correctly ret is True
        if not ret:
            break

        # Remove lines from the frame
        #frame = apply_threshold(frame, threshold_value)
        frame_without_lines = remove_lines(frame, threshold_lines, min_line_length, max_line_gap)

        # Display the frame without lines
        cv.imshow('frame', frame_without_lines)

        # Wait for 25 ms before moving on to the next frame
        # This will slow down the video
        cv.waitKey(25)

    # Release the video file
    cap.release()

    # Close all OpenCV windows
    cv.destroyAllWindows()

In [58]:
play_video_with_line_removal(path + files[0], 50, 120, 10, 180)

In [43]:
play_video_with_directional_filtering(path + files[0], 50)

In [52]:
play_video_with_thresholding(path + files[0], 180, 205)

In [26]:
play_video(path + files[0])

In [ ]:
get_all_frames(path + files[0])

In [4]:
frame = get_frame(path + files[0], 100)

In [6]:
cv.imshow('frame', frame)

: 